# Distributed RDataFrame

An `RDataFrame` analysis written in Python can be executed both *locally* - possibly in parallel on the cores of the machine - and *distributedly* by offloading computations to external resources, which include:

- [Spark](https://spark.apache.org/) and 
- [Dask](https://dask.org/) clusters. 

- This feature is enabled by the architecture depicted below.

- It shows that RDataFrame computation graphs can be mapped to different kinds of resources via backends.

- In this notebook we will exercise the Dask backend, which divides an `RDataFrame` input dataset in logical ranges and submits computations for each of those ranges to Dask resources.

<img src="../../images/DistRDF_architecture.png" alt="Distributed RDataFrame">

## Create a Dask client

- In order to work with a Dask cluster we need a `Client` object.
- It represents the connection to that cluster and allows to configure execution-related parameters (e.g. number of cores, memory). 
- The client object is just the intermediary between our client session and the cluster resources, 
- Dask supports many different resource managers.
- We will follow the [Dask documentation](https://distributed.dask.org/en/stable/client.html) regarding the creation of a `Client`.

In [ ]:
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=2, threads_per_worker=1, processes=True, memory_limit="2GiB")
client = Client(cluster)

## Create a ROOT dataframe

We now create an RDataFrame based on the same dataset seen in the exercise [rdataframe-dimuon](exercises/rdataframe-dimuon.ipynb).

A Dask `RDataFrame` receives two extra parameters: 
- the number of partitions to apply to the dataset (`npartitions`)
- the `Client` object (`daskclient`). 

Besides this detail, a Dask `RDataFrame` is not different from a local `RDataFrame`: the analysis presented in this notebook would not change if we wanted to execute it locally.

In [ ]:
# Use a Dask RDataFrame
RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame

df = RDataFrame("h42",
                "https://root.cern/files/h1big.root",
                npartitions=4,
                daskclient=client)

## Run your analysis unchanged

- From now on, the rest of your application can be written **exactly** as we have seen with local RDataFrame. 

- The goal of the distributed RDataFrame module is to support all the traditional RDataFrame operations (those that make sense in a distributed context at least). 

- Currently only a subset of those is available and can be found in the corresponding [section of the documentation](https://root.cern/doc/master/classROOT_1_1RDataFrame.html#distrdf)

In [ ]:
%%time
df1 = df.Filter("nevent > 1")
df2 = df1.Define("mpt","sqrt(xpt*xpt + ypt*ypt)")
c = df.Count()
m = df2.Mean("mpt")
print(f"Number of events after processing: {c.GetValue()}")
print(f"Mean of column 'mpt': {m.GetValue()}")